In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp

from general_ode_solver import FitzHughNagumoModel, NumericalSolver

#TODO How to test here: from data_generation.simulations import simulator as s

## Test Usage FitzHughNagumo Model with Solver

In [7]:
fhnM = FitzHughNagumoModel(control_params=['I'])
fhnM.get_config()

{'model': 'FitzHughNagumoModel',
 'x_dim': 2,
 'control_dim': 1,
 'control_params': ['I'],
 'a': 0.7,
 'b': 1.6,
 'epsilon': 0.08,
 'I': None}

In [8]:
control_paramsdict = {'I': np.array([0.35, 0.6])} 
control = np.array([[[0.35], [0.6]],[[0.35], [0.6]]]) #shape(num_steps, n_samples, control_dim)

t = 0
Z = np.array([1,0, 0, 1])
X = np.array([[1,0],[0,1]])
fhnM.odes_vectorized(t, Z, control_paramsdict)

array([ 1.01666667, -0.4       ,  0.136     , -0.072     ])

In [9]:
solver = NumericalSolver(fhnM)
solver.get_derivative(X, control_paramsdict)

array([[ 1.01666667,  0.136     ],
       [-0.4       , -0.072     ]])

In [10]:
num_steps = 2
e =solver.step(X, control, 0.5, num_steps)
e

array([[[ 1.        ,  0.        ],
        [ 0.        ,  1.        ]],

       [[ 1.44907584,  0.07517142],
        [-0.24755332,  0.96060908]],

       [[ 1.68892076,  0.15919073],
        [-0.61105003,  0.91186921]]])

## First Experiments FHN Model

In [27]:
def fhn(t, z, a, b, epsilon, I):
    v, w = z
    dvdt = v - (v**3) / 3 - w + I
    dwdt = epsilon * (v + a - b * w)
    return [dvdt, dwdt]

# Parameters
a = 0.7
b = 1.6
epsilon = 0.08

# Bifurcation parameter values for different regimes
I_values = [0.2, 0.35, 1.2]  # Monostable, Bistable, Oscillatory

# Time span for integration
t_span = (0, 100)
t_eval = np.linspace(t_span[0], t_span[1], 1000)

# Initial conditions for phase space trajectories
in_con = [
    [-1.5, -1], [0.5, 0], [1.5, 1]  # Different starting points in phase space
]



In [28]:
random_array = np.random.rand(100, 2)

In [29]:
sol_for_loop = []
for initcon in random_array:
    sol = solve_ivp(fhn, t_span, initcon, args=(a, b, epsilon, I_values[0]), t_eval=t_eval)
    sol_for_loop.append(sol)
       

In [30]:
def ode_vectorized(t, Z, a, b, epsilon, I):
    v, w = Z[:int(len(Z)/2)], Z[int(len(Z)/2):]  
    I_v = np.full_like(v, I)
    a_v = I = np.full_like(w, a)
    dvdt = v - (v**3) / 3 - w + I_v
    dwdt = epsilon * (v + a - b * w)
    return np.append(dvdt, dwdt)  # Return as a 1D array



In [31]:
initcon = random_array.transpose().flatten()
sol_vect = solve_ivp(ode_vectorized, t_span, initcon, args=(a, b, epsilon, I_values[0]), t_eval=t_eval)

In [33]:
sol_for_loop[0].y

array([[ 0.23670812,  0.2485576 ,  0.26120211, ..., -1.29475531,
        -1.29474509, -1.29473516],
       [ 0.31770856,  0.32116384,  0.32467245, ..., -0.3713519 ,
        -0.37135659, -0.37136113]])

In [43]:
sol_vect.y[int(len(sol_vect.y)/2)]

array([ 0.08887935,  0.10078892,  0.11315811,  0.12596617,  0.13918423,
        0.15277761,  0.16670576,  0.18092229,  0.19537487,  0.21000352,
        0.22475982,  0.23960157,  0.25448724,  0.26937599,  0.28422765,
        0.29900274,  0.31366613,  0.32820153,  0.34259749,  0.35684213,
        0.37092328,  0.38482841,  0.39854463,  0.41206008,  0.42537419,
        0.43848684,  0.45139699,  0.4641036 ,  0.47660559,  0.48890188,
        0.50099133,  0.51287483,  0.52455473,  0.53603266,  0.54731027,
        0.5583892 ,  0.56927108,  0.57995754,  0.59045057,  0.60075254,
        0.6108657 ,  0.62079228,  0.63053444,  0.64009428,  0.64947383,
        0.6586752 ,  0.66770071,  0.67655265,  0.68523323,  0.69374461,
        0.7020889 ,  0.71026813,  0.71828432,  0.72613961,  0.73383611,
        0.74137589,  0.74876094,  0.75599323,  0.76307466,  0.77000708,
        0.77679245,  0.78343265,  0.78992954,  0.79628493,  0.80250058,
        0.80857822,  0.8145196 ,  0.82032644,  0.82600041,  0.83

In [44]:
sol_vect.y[0]-sol_for_loop[0].y[0]

array([ 0.00000000e+00,  6.00920061e-06,  4.60960192e-05,  9.99355728e-05,
        1.66234221e-04,  2.29180702e-04,  2.58446135e-04,  2.09184018e-04,
        2.27552697e-05, -2.42845965e-04, -3.59418532e-04, -2.24040127e-04,
       -1.83685649e-05,  9.05318955e-05,  6.70517582e-05, -4.49877589e-05,
       -1.12699116e-04, -6.39661371e-05,  3.72978800e-05,  9.44810283e-05,
        1.18678705e-04,  1.24619346e-04,  1.18847693e-04,  1.07804369e-04,
        1.18007887e-04,  9.22243621e-05,  1.12194963e-05, -8.73481170e-05,
       -1.73485552e-04, -2.25040413e-04, -2.27739148e-04, -1.86649000e-04,
       -1.24806478e-04, -6.46607262e-05, -3.00562840e-05, -3.67169747e-05,
       -6.54308991e-05, -9.93894702e-05, -1.28635044e-04, -1.49459084e-04,
       -1.59453010e-04, -1.57435092e-04, -1.43450451e-04, -1.18771056e-04,
       -8.58957271e-05, -4.94241552e-05, -1.79825440e-05, -1.09704151e-06,
       -2.50999462e-06, -1.32678289e-05, -3.09529615e-05, -5.32938653e-05,
       -7.81814492e-05, -